### Module 11.Demo 11 : CNN 

In [ ]:
#### MNIST =============================
import torch
from torchvision.datasets import mnist
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import torch.nn.functional as F
from torch import nn
import matplotlib.pyplot as plt
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
transform = transforms.Compose(
    [transforms.ToTensor(),transforms.Normalize((0.5), (0.5))])
trainset = mnist.MNIST('./dataset/mnist', train=True, transform=transform, download=True)
testset = mnist.MNIST('./dataset/mnist', train = False, transform=transform, download=True)
train_data = DataLoader(trainset, batch_size=64, shuffle=True)
test_data = DataLoader(testset, batch_size=128, shuffle=False)
for im, label in train_data:
    print(im.shape);  break

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0) #output_shape=(16,24,24)
        self.relu1 = nn.ReLU() # activation
        self.maxpool1 = nn.MaxPool2d(kernel_size=2) #output_shape=(16,12,12)#Max pool 1
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0) #output_shape=(32,8,8)
        self.relu2 = nn.ReLU() # activation
        self.maxpool2 = nn.MaxPool2d(kernel_size=2) #output_shape=(32,4,4) # Max pool 2
        # Fully connected 1 ,#input_shape=(32*4*4)
        self.fc1 = nn.Linear(32 * 4 * 4, 10)
    def forward(self, x):
        # Convolution 1
        x = self.cnn1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)  # Max pool 1
        x = self.cnn2(x) # Convolution 2 
        x = self.relu2(x)
        x = self.maxpool2(x)  # Max pool 2 
        x = x.view(x.size(0), -1)
        # Linear function (readout)
        x = self.fc1(x) ; return x
model = CNN().to(device)  # build model
criterion = nn.CrossEntropyLoss()  # define cost
optimizer = torch.optim.SGD(model.parameters(), 0.1) # optimization
#optimizer = torch.optim.RMSprop(model.parameters(), 0.01, alpha=0.9)
#optimizer = torch.optim.Adam(model.parameters(), 0.01, betas=(0.9, 0.99))
losses = [] ; acces = [] ; eval_losses = [] ;eval_acces = []
for epoch in range(5):
    train_loss = 0 ;    train_acc = 0 ;    model.train()      
    for im, label in train_data:
        im = im.to(device)
        label = label.to(device)
        pred = model(im)
        loss = criterion(pred, label)
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, pred = pred.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        train_acc += acc
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    eval_loss = 0 ;    eval_acc = 0   ;    model.eval()
    for im, label in test_data:
        im = im.to(device)
        label = label.to(device)
        pred = model(im)
        loss = criterion(pred, label)
        eval_loss += loss.item()
        _, pred = pred.max(1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / im.shape[0]
        eval_acc += acc
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    print('epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}, Eval Loss: {:.6f}, Eval Acc: {:.6f}'
          .format(epoch, train_loss / len(train_data), train_acc / len(train_data), eval_loss / len(test_data), eval_acc / len(test_data)))

In [ ]:
plt.plot(np.arange(0,5),losses)
plt.plot(np.arange(0,5),eval_losses)
plt.title('Model  Loss');plt.xlabel('Epoch')  ; plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper right'); plt.show()

In [ ]:
plt.plot(np.arange(0,5),acces)
plt.plot(np.arange(0,5),eval_acces)
plt.title('Model  Accuracy')
plt.xlabel('Epoch') ;  plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper right') ;plt.show()

In [ ]:
### Cifar10 ========================================
import torch
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=True,
                                        download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                            shuffle=True, num_workers=2)  # num_workers=2: CPU開雙核心
testset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0))) ;  plt.show() # pytorch 深度要放最後
# get some random training images
dataiter = iter(trainloader) ; images, labels = dataiter.next()
# show images : torchvision.utils.make_grid(images) show 一堆image
imshow(torchvision.utils.make_grid(images))
print(' '.join('%5s' % classes[labels[j]] for j in range(4))) # # print labels

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x);   return x
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data # # get the inputs; data is a list of [inputs, labels]
        optimizer.zero_grad()  # zero the parameter gradients  
        outputs = net(inputs)  # forward + backward + optimize
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()   # print statistics
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
print('Finished Training')

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)  # net.state_dict() : 只有存參數

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images)) # print images
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))

In [ ]:
outputs = net(images)
_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

In [ ]:
correct = 0 ;total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

### Cat and Dog Classification

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
class CustomImageDataset(Dataset):
    def read_data_set(self):
        all_img_files = []
        all_labels = []
        class_names = []
        for index, i in enumerate(os.listdir(self.data_set_path)):
            class_names.append(i)
            for j in os.listdir(self.data_set_path+'/'+i):
                all_img_files.append(self.data_set_path+'/'+i+'/'+j)
                all_labels.append(index)
        #print(all_img_files)
        #print(all_labels)
        return all_img_files, all_labels, len(all_img_files), len(class_names)
    def __init__(self, data_set_path, transforms=None):
        self.data_set_path = data_set_path
        #self.read_data_set2()
        self.image_files_path, self.labels, self.length, self.num_classes = self.read_data_set()
        self.transforms = transforms
    def __getitem__(self, index):
        image = Image.open(self.image_files_path[index])
        image = image.convert("RGB")
        if self.transforms is not None:
            image = self.transforms(image)
        label = self.labels[index]
        return image, label
    def __len__(self):
        return self.length
class CustomConvNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomConvNet, self).__init__()
        self.layer1 = self.conv_module(3, 16)
        self.layer2 = self.conv_module(16, 32)
        self.layer3 = self.conv_module(32, 64)
        self.layer4 = self.conv_module(64, 128)
        self.layer5 = self.conv_module(128, 256)
        self.gap = self.global_avg_pool(256, num_classes)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.gap(out)
        out = out.view(-1, num_classes)
        return out
    def conv_module(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
    def global_avg_pool(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.AdaptiveAvgPool2d((1, 1)))
hyper_param_epoch = 3
hyper_param_batch = 32
hyper_param_learning_rate = 0.001
transforms_train = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.RandomRotation(10.),
                                       transforms.ToTensor()])
transforms_test = transforms.Compose([transforms.Resize((128, 128)),
                                      transforms.ToTensor()])
train_data_set = CustomImageDataset(data_set_path="./data/train", transforms=transforms_train)
train_loader = DataLoader(train_data_set, batch_size=hyper_param_batch, shuffle=True)
test_data_set = CustomImageDataset(data_set_path="./data/test", transforms=transforms_test)
test_loader = DataLoader(test_data_set, batch_size=hyper_param_batch, shuffle=True)
num_classes = train_data_set.num_classes
custom_model = CustomConvNet(num_classes=num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(custom_model.parameters(), lr=hyper_param_learning_rate)
for e in range(hyper_param_epoch):
    for i_batch, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = custom_model(images)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i_batch + 1) % hyper_param_batch == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'
                  .format(e + 1, hyper_param_epoch, loss.item()))
# Test the model
custom_model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels  in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = custom_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total))  

### Cat and Dog Classification(ImageFolder)
####  ImageFolder API  for  image用

In [ ]:
import os
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
transforms_train = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.RandomRotation(10.),
                                       transforms.ToTensor()])
train_data_set = ImageFolder('./data/train', transform=transforms_train)
print(train_data_set.class_to_idx)
print(train_data_set.imgs[:5])
print(train_data_set[0][1])
print(train_data_set[0][0].shape)

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
from torchvision.datasets import ImageFolder
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
class CustomConvNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomConvNet, self).__init__()
        self.layer1 = self.conv_module(3, 16)
        self.layer2 = self.conv_module(16, 32)
        self.layer3 = self.conv_module(32, 64)
        self.layer4 = self.conv_module(64, 128)
        self.layer5 = self.conv_module(128, 256)
        self.gap = self.global_avg_pool(256, num_classes)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.gap(out)
        out = out.view(-1, num_classes)
        return out
    def conv_module(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
    def global_avg_pool(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.AdaptiveAvgPool2d((1, 1)))
hyper_param_epoch = 3
hyper_param_batch = 32
hyper_param_learning_rate = 0.001
transforms_train = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.RandomRotation(10.),
                                       transforms.ToTensor()])
transforms_test = transforms.Compose([transforms.Resize((128, 128)),
                                      transforms.ToTensor()])
train_data_set = ImageFolder('./data/train', transform=transforms_train)
train_loader = DataLoader(train_data_set, batch_size=hyper_param_batch, shuffle=True)
test_data_set = ImageFolder('./data/test', transform=transforms_test)
test_loader = DataLoader(test_data_set, batch_size=hyper_param_batch, shuffle=True)
num_classes = len(train_data_set.class_to_idx)
custom_model = CustomConvNet(num_classes=num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(custom_model.parameters(), lr=hyper_param_learning_rate)
for e in range(hyper_param_epoch):
    for i_batch, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)        
        # Forward pass
        outputs = custom_model(images)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i_batch + 1) % hyper_param_batch == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'
                  .format(e + 1, hyper_param_epoch, loss.item()))
# Test the model
custom_model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels  in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = custom_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy on the {} test images: {} %'.format(total, 100 * correct / total))  